In [1]:
import json, requests
from datetime import datetime
from dateutil.relativedelta import relativedelta

headers = {
    'x-rapidapi-key': "689dad5751msh99cc2e93ffca412p124552jsnf5c96d920412",
    'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com"
    }


In [2]:
# manually set variables for our user
account_id = 1001
cash_amount = 1000.00
investment_account = {
    "GME": 20,
    "TSLA": 30,
    "TR": 300,
    "SPY": 75,
    "AMD": 10,
    "GOOG": 0.5,
    "AAPL": 30,
    "FB": 10,
    "NOK": 200,
    "AMC": 50,
    
}

In [3]:
# GENERATES ACCOUNT HISTORY AND INSTRUMENT DATA FOR SQL
account_history_sql = []
instruments_sql = []

for symbol in investment_account.keys():
    # go to api for historical data
    url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v3/get-historical-data"
    querystring = {"symbol": symbol,"region":"US"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    prices = json.loads(response.text)["prices"]
    
    # only want to save the relevant dates ie- past 3 months as well as the adjusted close prices
    save_dates = []
    today_price = None;
    for price in prices:
        if datetime.fromtimestamp(price["date"]).date() >= datetime.today().date() - relativedelta(months=3) and "adjclose" in price:
            # saving today's price for the instruments data in the next step
            if datetime.fromtimestamp(price["date"]).date() >= datetime.today().date():
                today_price = price["adjclose"]
            # generate sql statements for account history
            account_history_sql.append("INSERT INTO account_history VALUES({0}, {1}, {2});".format(account_id, str(datetime.fromtimestamp(price["date"]).date()), round(price["adjclose"] * investment_account[symbol] + cash_amount, 2)))
    
    # go to api for instrument summaries
    url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-summary"
    querystring = {"symbol": symbol,"region":"US"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    summary = json.loads(response.text)["quoteType"]
    
    # generate sql statements for instrument summaries
    if summary["quoteType"] == "EQUITY":
        instruments_sql.append("INSERT INTO instruments VALUES({0}, {1}, {2}, {3});".format(symbol, summary["longName"], "Stock", today_price))
    else:
        instruments_sql.append("INSERT INTO instruments VALUES({0}, {1}, {2}, {3});".format(symbol, summary["longName"], summary["quoteType"].lower().title(), today_price))   

In [4]:
file = open("insertData.sql", "a")
for statement in account_history_sql:
    file.write(statement + "\n")
file.write("\n")
for statement in instruments_sql:
    file.write(statement + "\n")
file.close()